In [1]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import text

import tensorflow as tf
assert tf.__version__.startswith('2')
# tf.get_logger().setLevel('ERROR')

/home/sun/prog/tg-prog-lang-cat/research/tf/tflite/.venv/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/sun/prog/tg-prog-lang-cat/research/tf/tflite/.venv/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not

In [2]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

In [3]:
import pandas as pd

def replace_label(original_file, new_file):
  # Load the original file to pandas. We need to specify the separator as
  # '\t' as the training data is stored in TSV format
  df = pd.read_csv(original_file, sep='\t')

  # Define how we want to change the label name
  label_map = {0: 'negative', 1: 'positive'}

  # Excute the label change
  df.replace({'label': label_map}, inplace=True)

  # Write the updated dataset to a new file
  df.to_csv(new_file)

# Replace the label name for both the training and test dataset. Then write the
# updated CSV dataset to the current folder.
replace_label(os.path.join(os.path.join(data_dir, 'train.tsv')), 'train.csv')
replace_label(os.path.join(os.path.join(data_dir, 'dev.tsv')), 'dev.csv')

In [4]:
mb_spec = model_spec.get('mobilebert_classifier')

2023-11-14 15:05:24.512973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-14 15:05:24.555823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-14 15:05:24.556079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [5]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

2023-11-14 15:05:50.294346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 15:05:50.295800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-14 15:05:50.296509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-14 15:05:50.296729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.config.set_logical_device_configuration

<function tensorflow.python.framework.config.set_logical_device_configuration(device, logical_devices)>

In [8]:
model = text_classifier.create?

Signature:
text_classifier.create(
    train_data,
    model_spec='average_word_vec',
    validation_data=None,
    batch_size=None,
    epochs=3,
    steps_per_epoch=None,
    shuffle=False,
    do_train=True,
)
Docstring:
Loads data and train the model for test classification.

Args:
  train_data: Training data.
  model_spec: Specification for the model.
  validation_data: Validation data. If None, skips validation process.
  batch_size: Batch size for training.
  epochs: Number of epochs for training.
  steps_per_epoch: Integer or None. Total number of steps (batches of
    samples) before declaring one epoch finished and starting the next
    epoch. If `steps_per_epoch` is None, the epoch will run until the input
    dataset is exhausted.
  shuffle: Whether the data should be shuffled.
  do_train: Whether to run training.

Returns:
  An instance based on TextClassifier.
File:      ~/prog/tg-prog-lang-cat/research/tf/tflite/.venv/lib/python3.9/site-packages/tensorflow_examples/lite/

In [9]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3, shuffle=True, batch_size=8)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


8418/8418 [==============================] - 4533s 531ms/step - loss: 0.3912 - test_accuracy: 0.8860
Epoch 2/2
8418/8418 [==============================] - 4466s 530ms/step - loss: 0.1547 - test_accuracy: 0.9614
Epoch 3/3
8418/8418 [==============================] - 4481s 532ms/step - loss: 0.0854 - test_accuracy: 0.9808


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 hub_keras_layer_v1v2 (HubKeras  (None, 512)         24581888    ['input_word_ids[0][0]',         
 LayerV1V2)                                                       'input_mask[0][0]',         

In [11]:
loss, acc = model.evaluate(test_data)

28/28 [==============================] - 24s 623ms/step - loss: 0.4501 - test_accuracy: 0.9209


In [12]:
model.export(export_dir='mobilebert-default/')

2023-11-14 18:51:26.566015: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpum2wz7am/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpum2wz7am/saved_model/assets
2023-11-14 18:52:00.865952: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-14 18:52:00.865981: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-14 18:52:00.867377: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpum2wz7am/saved_model
2023-11-14 18:52:00.916730: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-14 18:52:00.916763: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpum2wz7am/saved_model
2023-11-14 18:52:01.124872: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-14 18:52:04.777890: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpum2wz7am/saved_model
2023-11-14 18:52:07.285839: I ten

INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['vocab.txt', 'labels.txt']


INFO:tensorflow:['vocab.txt', 'labels.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert-default/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert-default/model.tflite


In [13]:
model.export(export_dir='mobilebert-label-vocab/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

INFO:tensorflow:Saved vocabulary in mobilebert-label-vocab/vocab.txt.


INFO:tensorflow:Saved vocabulary in mobilebert-label-vocab/vocab.txt.


INFO:tensorflow:Saving labels in mobilebert-label-vocab/labels.txt


INFO:tensorflow:Saving labels in mobilebert-label-vocab/labels.txt
